In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

In [2]:
data = pd.read_csv(r'C:\Users\amina\OneDrive\Desktop\companies-data.csv')

In [3]:
data.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [4]:
data.dtypes

Bankrupt?                                                     int64
 ROA(C) before interest and depreciation before interest    float64
 ROA(A) before interest and % after tax                     float64
 ROA(B) before interest and depreciation after tax          float64
 Operating Gross Margin                                     float64
                                                             ...   
 Liability to Equity                                        float64
 Degree of Financial Leverage (DFL)                         float64
 Interest Coverage Ratio (Interest expense to EBIT)         float64
 Net Income Flag                                              int64
 Equity to Liability                                        float64
Length: 96, dtype: object

In [5]:
for i in list(data.columns):
    if data[i].std() == 0:
        print(i)

 Net Income Flag


In [6]:
data = data.drop(' Net Income Flag', axis=1)

In [7]:
data['Bankrupt?'].value_counts()

0    6599
1     220
Name: Bankrupt?, dtype: int64

In [8]:
X = data.drop('Bankrupt?', axis=1).values
y = data['Bankrupt?'].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [10]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
pca = PCA(n_components=0.9).fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [11]:
X_train.shape

(5455, 45)

In [12]:
log_model = LogisticRegressionCV(class_weight='balanced', scoring='precision', cv=5)

In [13]:
log_model.fit(X_train, y_train)

C:\Users\amina\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\amina\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

LogisticRegressionCV(class_weight='balanced', cv=5, scoring='precision')

In [14]:
from sklearn.metrics import confusion_matrix

In [15]:
confusion_matrix(y_test, log_model.predict(X_test))

array([[1152,  168],
       [  11,   33]], dtype=int64)

In [16]:
rf_model = RandomForestClassifier(n_estimators=200, n_jobs=-1, class_weight='balanced')

In [17]:
params = {
    'max_depth':[2,4,6,8],
    'min_samples_split':[2,4,6,8],
    'min_samples_leaf':[1,3,5,7]
}

In [18]:
rf_tuner = RandomizedSearchCV(rf_model, params, n_iter=20, scoring='recall', cv=5)

In [19]:
rf_tuner.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=RandomForestClassifier(class_weight='balanced',
                                                    n_estimators=200,
                                                    n_jobs=-1),
                   n_iter=20,
                   param_distributions={'max_depth': [2, 4, 6, 8],
                                        'min_samples_leaf': [1, 3, 5, 7],
                                        'min_samples_split': [2, 4, 6, 8]},
                   scoring='recall')

In [20]:
confusion_matrix(y_test, rf_tuner.predict(X_test))

array([[1180,  140],
       [  11,   33]], dtype=int64)

In [21]:
input = keras.layers.Input(shape=(X_train.shape[1],))
x = keras.layers.Dense(32, activation='relu')(input)
x = keras.layers.BatchNormalization()(x)
output = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.models.Model(input, output)

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.0001), metrics=[keras.metrics.Recall(name='recall')])

In [22]:
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(np.unique(y))
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_dict = {}
for i in classes:
    class_dict[i] = class_weights[i]

In [23]:
class myCallback(keras.callbacks.Callback):
    best = 0
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_recall') > self.best:
            self.best = logs.get('val_recall')
            model.save('bestmodel.hdf5')
        print(f'\nbest recall is : {self.best}')
mc = myCallback()

In [24]:
from keras.callbacks import ReduceLROnPlateau
callbacks = [ReduceLROnPlateau(patience=2), mc]

In [25]:
model.fit(X_train, y_train, validation_split=0.2, batch_size=32, epochs=100, callbacks=callbacks, class_weight=class_dict)

Epoch 1/100
112/137 [=======================>......] - ETA: 0s - loss: 0.5507 - recall: 0.8783
best recall is : 0.875
137/137 [==============================] - 1s 4ms/step - loss: 0.5426 - recall: 0.8889 - val_loss: 0.9492 - val_recall: 0.8750
Epoch 2/100
115/137 [========================>.....] - ETA: 0s - loss: 0.5282 - recall: 0.8760
best recall is : 0.875
137/137 [==============================] - 0s 2ms/step - loss: 0.5209 - recall: 0.8889 - val_loss: 0.7855 - val_recall: 0.8125
Epoch 3/100
116/137 [========================>.....] - ETA: 0s - loss: 0.4894 - recall: 0.9055
best recall is : 0.875
137/137 [==============================] - 0s 2ms/step - loss: 0.4858 - recall: 0.9097 - val_loss: 0.7327 - val_recall: 0.8125
Epoch 4/100
113/137 [=======================>......] - ETA: 0s - loss: 0.4667 - recall: 0.9009
best recall is : 0.875
137/137 [==============================] - 0s 2ms/step - loss: 0.4718 - recall: 0.9028 - val_loss: 0.6930 - val_recall: 0.8125
Epoch 5/100
111/137 

In [26]:
model.load_weights('bestmodel.hdf5')
confusion_matrix(y_test, np.round(model.predict(X_test)))

array([[736, 584],
       [  4,  40]], dtype=int64)